In [1]:
import ipywidgets as w

from utils.wavefront import generator, writer

In [2]:
model = None

In [ ]:
def header(x: str, level=1) -> w.HTML:
    return w.HTML(f"<h{level}>{x}</h{level}>")


def text(x: str) -> w.HTML:
    return w.HTML(f"<p>{x}</p>")


a = w.Dropdown(options=[1, 2, 3])
b = w.VBox([header("Test"), a])
b

In [ ]:
children = [header("Select your writer")]
tabs = []
for writ in writer.__all__:
    num_optinal_parameters = len(writer.__dict__[writ].__annotations__) - 2
    if num_optinal_parameters >= 2:
        text_folder = w.Text(placeholder="folder", value="multi_tmp")
        text_file = w.Text(value="tmp_{i}.obj", placeholder="file")
    elif num_optinal_parameters >= 1:
        text_file = w.Text(value="tmp.obj", placeholder="file")

    fn = writer.__dict__[writ]

    def write_to(_):
        print(tab.children[3].value, "from", writ)
        fn(model, folder_name=text_folder.value, file_name=text_file.value)

    tab = w.VBox(
        [
            header(writ, 2),
            text(fn.__doc__),
        ]
    )

    if num_optinal_parameters >= 2:
        tab.children += (text_folder,)
    tab.children = [*tab.children, text_file, b]
    tabs.append(((writ, tab)))
    del tab
# writer.__dict__["write_multi_part_obj_file"]
t = w.Tab()
t.children = [t for _, t in tabs]
t.titles = [t for t, _ in tabs]
children.append(t)
b = w.Button(description="Create model")


def get_child(i: int, id: str):
    c = [
        c
        for c in t.children[i].children
        if "placeholder" in dir(c)
        and ord(c.placeholder[0]) <= 255
        and c.placeholder == id
    ]


def func(values):
    if values["name"] == "selected_index":
        cs = [
            c
            for c in t.children[values["new"]].children
            if "placeholder" in dir(c) and ord(c.placeholder[0]) <= 255
        ]
        if len(cs) == 1:

            def func(_):
                print(f"calling with {cs[0].value}")
                # writer.__dict__[writer.__all__[values["new"]]](
                #     model, file_name=cs[0].value
                # )

        else:
            cs = sorted(cs, key=lambda x: x.placeholder)

            def func(_):
                print(f"calling with {cs[0].value} and {cs[1].value}")
                # writer.__dict__[writer.__all__[values["new"]]](
                #     model, file_name=cs[0].value, folder_name=cs[1].value
                # )

        b.on_click(func, remove=True)


t.observe(func, "selected_index")


# t._selected_index_changed = func
t

In [ ]:
print(t.selected_index)
dir(b)

In [ ]:
print(a.value)